In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import os
import glob

<IPython.core.display.Javascript object>

In [3]:
!ls grouped

BidirectionalGRU_results_grouped_1.csv	  lstm_results_grouped_1.csv
BidirectionalGRU_results_grouped_10.csv   lstm_results_grouped_10.csv
BidirectionalGRU_results_grouped_2.csv	  lstm_results_grouped_2.csv
BidirectionalGRU_results_grouped_3.csv	  lstm_results_grouped_3.csv
BidirectionalGRU_results_grouped_4.csv	  lstm_results_grouped_4.csv
BidirectionalGRU_results_grouped_5.csv	  lstm_results_grouped_5.csv
BidirectionalGRU_results_grouped_6.csv	  lstm_results_grouped_6.csv
BidirectionalGRU_results_grouped_7.csv	  lstm_results_grouped_7.csv
BidirectionalGRU_results_grouped_8.csv	  lstm_results_grouped_8.csv
BidirectionalGRU_results_grouped_9.csv	  lstm_results_grouped_9.csv
BidirectionalLSTM_results_grouped_1.csv   mlp_results_grouped_1.csv
BidirectionalLSTM_results_grouped_10.csv  mlp_results_grouped_10.csv
BidirectionalLSTM_results_grouped_2.csv   mlp_results_grouped_2.csv
BidirectionalLSTM_results_grouped_3.csv   mlp_results_grouped_3.csv
BidirectionalLSTM_results_grouped_4.csv   mlp

<IPython.core.display.Javascript object>

### Reading The files

In [5]:
path = "/home/peressim/projects/ccs28-ml-modelling/reports/results/local_models/204/f/grouped/"
csv_files = glob.glob(os.path.join(path, "*.csv"))

<IPython.core.display.Javascript object>

In [6]:
results = []

for filepath in csv_files:
    df = pd.read_csv(filepath, header=[0, 1])
    results.append(df)
df = pd.concat(results, ignore_index=True)

<IPython.core.display.Javascript object>

### Renaming the Columns

In [7]:
df.columns = pd.MultiIndex.from_tuples(
    list(
        {
            col: (col[0], "") if "Unnamed" in col[1] else col for col in df.columns
        }.values()
    )
)

<IPython.core.display.Javascript object>

### Defining the Columns to work with

In [8]:
cols = [
    ("Model", ""),
    ("Features", ""),
    ("Cross Validation", ""),
    ("Cross Validation Params", ""),
    ("RMSE Test", "mean"),
    ("RMSE Test", "std"),
    ("MAE Test", "mean"),
    ("MAE Test", "std"),
    ("MAPE Test", "mean"),
    ("MAPE Test", "std"),
    ("R2 Test", "mean"),
    ("R2 Test", "std"),
]

<IPython.core.display.Javascript object>

In [9]:
cols_metrics = [
    ("RMSE Test", "mean"),
    ("RMSE Test", "std"),
    ("MAE Test", "mean"),
    ("MAE Test", "std"),
    ("MAPE Test", "mean"),
    ("MAPE Test", "std"),
    ("R2 Test", "mean"),
    ("R2 Test", "std"),
]

<IPython.core.display.Javascript object>

# Linear Regression

In [10]:
df_linreg = df[df[("Model", "")] == "Linear Regression"][cols].copy()
# df_linreg = df_linreg.drop([("Model", "")], axis=1)
df_linreg = df_linreg[df_linreg["Cross Validation"] != "Out of time Split"]

<IPython.core.display.Javascript object>

In [11]:
df_linreg[df_linreg["Features"] == "Chemical"]

Model  Features            Cross Validation  \
                                                               
284  Linear Regression  Chemical  Blocking Time Series Split   
286  Linear Regression  Chemical              Repeated KFold   
287  Linear Regression  Chemical           Time Series Split   

                              Cross Validation Params RMSE Test            \
                                                           mean       std   
284  {"N_Splits": 5, "Repeats": 1, "train_size": 0.8}  1.998465  0.484797   
286                     {"N_Splits": 5, "Repeats": 3}  1.969420  0.152776   
287           {"N_Splits": 5, "Repeats": 1, "Gap": 0}  2.135229  0.315449   

     MAE Test           MAPE Test             R2 Test            
         mean       std      mean       std      mean       std  
284  1.517878  0.289553  0.035482  0.006823  0.909912  0.044190  
286  1.410058  0.039831  0.032871  0.001115  0.919417  0.012880  
287  1.548382  0.172280  0.036036  0.003653  0.902621  0.028284

<IPython.core.display.Javascript object>

In [12]:
df_linreg[df_linreg["Features"] == "Chemical"].drop(["Cross Validation Params"], axis=1)

/tmp/ipykernel_1879735/2542665763.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_linreg[df_linreg["Features"] == "Chemical"].drop(["Cross Validation Params"], axis=1)


Model  Features            Cross Validation RMSE Test  \
                                                                  mean   
284  Linear Regression  Chemical  Blocking Time Series Split  1.998465   
286  Linear Regression  Chemical              Repeated KFold  1.969420   
287  Linear Regression  Chemical           Time Series Split  2.135229   

               MAE Test           MAPE Test             R2 Test            
          std      mean       std      mean       std      mean       std  
284  0.484797  1.517878  0.289553  0.035482  0.006823  0.909912  0.044190  
286  0.152776  1.410058  0.039831  0.032871  0.001115  0.919417  0.012880  
287  0.315449  1.548382  0.172280  0.036036  0.003653  0.902621  0.028284

<IPython.core.display.Javascript object>

In [15]:
idx_min = df_linreg[df_linreg["Features"] == "Chemical"]["RMSE Test", "mean"].idxmin()

df_linreg.loc[[idx_min]].round(3)

Model  Features Cross Validation  \
                                                    
286  Linear Regression  Chemical   Repeated KFold   

           Cross Validation Params RMSE Test        MAE Test       MAPE Test  \
                                        mean    std     mean   std      mean   
286  {"N_Splits": 5, "Repeats": 3}     1.969  0.153     1.41  0.04     0.033   

           R2 Test         
       std    mean    std  
286  0.001   0.919  0.013

<IPython.core.display.Javascript object>

In [20]:
exclude = r"(?!.*One-Hot)"
pattern1 = r"Chemical \+ CS*"
pattern2 = r"Chemical \+ Physical"
pattern = "(" + pattern1 + "|" + pattern2 + ")" + exclude

features_filter = df_linreg["Features"].str.contains(
    pattern,
    regex=True,
)

idx_min = df_linreg[features_filter].round(3)["RMSE Test", "mean"].idxmin()

df_linreg.loc[[idx_min]].round(3)

/tmp/ipykernel_1879735/2110954779.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  features_filter = df_linreg["Features"].str.contains(


Model             Features Cross Validation  \
                                                               
427  Linear Regression  Chemical + Physical   Repeated KFold   

           Cross Validation Params RMSE Test        MAE Test        MAPE Test  \
                                        mean    std     mean    std      mean   
427  {"N_Splits": 5, "Repeats": 3}     1.155  0.038    0.896  0.025     0.021   

           R2 Test         
       std    mean    std  
427  0.001   0.972  0.002

<IPython.core.display.Javascript object>

In [19]:
df_linreg[df_linreg["Features"] == "Chemical + Properties CS Less"].drop(
    ["Cross Validation Params"], axis=1
)

/tmp/ipykernel_1879735/3721632251.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_linreg[df_linreg["Features"] == "Chemical + Properties CS Less"].drop(


Model                       Features  \
                                                        
770  Linear Regression  Chemical + Properties CS Less   
772  Linear Regression  Chemical + Properties CS Less   
773  Linear Regression  Chemical + Properties CS Less   

               Cross Validation RMSE Test            MAE Test            \
                                     mean       std      mean       std   
770  Blocking Time Series Split  1.615942  0.255376  1.251434  0.230804   
772              Repeated KFold  1.671293  0.069637  1.227671  0.030178   
773           Time Series Split  1.878567  0.332742  1.375587  0.215586   

    MAPE Test             R2 Test            
         mean       std      mean       std  
770  0.029164  0.005407  0.942482  0.018478  
772  0.028706  0.000831  0.942246  0.004960  
773  0.031726  0.004659  0.923713  0.024823

<IPython.core.display.Javascript object>

In [18]:
df_linreg[df_linreg["Features"] == "Chemical + Physical"].drop(
    ["Cross Validation Params"], axis=1
)

/tmp/ipykernel_1879735/2396273699.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_linreg[df_linreg["Features"] == "Chemical + Physical"].drop(


Model             Features            Cross Validation  \
                                                                          
425  Linear Regression  Chemical + Physical  Blocking Time Series Split   
427  Linear Regression  Chemical + Physical              Repeated KFold   
428  Linear Regression  Chemical + Physical           Time Series Split   

    RMSE Test            MAE Test           MAPE Test             R2 Test  \
         mean       std      mean       std      mean       std      mean   
425  1.251445  0.223054  0.980931  0.184659  0.022820  0.004179  0.965662   
427  1.155319  0.037970  0.896347  0.024984  0.020684  0.000550  0.972421   
428  1.301746  0.256804  1.004406  0.172853  0.023288  0.004055  0.962958   

               
          std  
425  0.010835  
427  0.001843  
428  0.015233

<IPython.core.display.Javascript object>

# Neural Networks

In [21]:
neural_networks = [
    "MLP",
    "LSTM",
    "GRU",
    "BidirectionalGRU",
    "BidirectionalLSTM",
    "Transformer",
]

df_nn = df[df[("Model", "")].isin(neural_networks)][cols].copy()
# df_nn = df_nn.drop([("Model", "")], axis=1)
df_nn = df_nn[df_nn["Cross Validation"] != "Out of time Split"]

<IPython.core.display.Javascript object>

In [22]:
idx_min = df_nn[df_nn["Features"] == "Chemical"]["RMSE Test", "mean"].idxmin()

df_nn.drop(["Cross Validation Params"], axis=1).loc[[idx_min]].round(3)

/tmp/ipykernel_1879735/558007381.py:3: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_nn.drop(["Cross Validation Params"], axis=1).loc[[idx_min]].round(3)


Model  Features Cross Validation RMSE Test        MAE Test        \
                                          mean    std     mean   std   
378  LSTM  Chemical   Repeated KFold     1.458  0.059    1.144  0.05   

    MAPE Test        R2 Test         
         mean    std    mean    std  
378     0.026  0.001   0.955  0.004

<IPython.core.display.Javascript object>

In [23]:
exclude = r"(?!.*One-Hot)"
pattern1 = r"Chemical \+ CS*"
pattern2 = r"Chemical \+ Physical"
pattern = "(" + pattern1 + "|" + pattern2 + ")" + exclude

# x = df_nn["Features"] != "Chemical + Mineralogical + Physical"
features_filter = df_nn["Features"].str.contains(
    pattern,
    regex=True,
)

idx_min = df_nn[features_filter].round(3)["RMSE Test", "mean"].idxmin()

df_nn.drop(["Cross Validation Params"], axis=1).loc[[idx_min]].round(3)

/tmp/ipykernel_1879735/1869263897.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  features_filter = df_nn["Features"].str.contains(
/tmp/ipykernel_1879735/1869263897.py:14: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_nn.drop(["Cross Validation Params"], axis=1).loc[[idx_min]].round(3)


Model              Features Cross Validation RMSE Test         \
                                                             mean    std   
491  Transformer  Chemical + CS3 + CS7   Repeated KFold     1.097  0.018   

    MAE Test        MAPE Test      R2 Test         
        mean    std      mean  std    mean    std  
491     0.86  0.014      0.02  0.0   0.975  0.001

<IPython.core.display.Javascript object>

In [24]:
idx_min = df_nn[df_nn["Features"] == "Chemical + Properties CS Less"][
    "RMSE Test", "mean"
].idxmin()

df_nn.drop(["Cross Validation Params"], axis=1).loc[[idx_min]].round(3)

/tmp/ipykernel_1879735/1533320894.py:5: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_nn.drop(["Cross Validation Params"], axis=1).loc[[idx_min]].round(3)


Model                       Features Cross Validation RMSE Test  \
                                                                      mean   
668  Transformer  Chemical + Properties CS Less   Repeated KFold     1.311   

           MAE Test        MAPE Test        R2 Test         
       std     mean    std      mean    std    mean    std  
668  0.036    1.037  0.031     0.024  0.001   0.965  0.002

<IPython.core.display.Javascript object>

In [25]:
idx_min = df_nn[df_nn["Features"] == "Chemical + Physical"][
    "RMSE Test", "mean"
].idxmin()

df_nn.drop(["Cross Validation Params"], axis=1).loc[[idx_min]].round(3)

/tmp/ipykernel_1879735/2784722090.py:5: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_nn.drop(["Cross Validation Params"], axis=1).loc[[idx_min]].round(3)


Model             Features Cross Validation RMSE Test        \
                                                                 mean   std   
154  BidirectionalGRU  Chemical + Physical   Repeated KFold     1.112  0.03   

    MAE Test        MAPE Test        R2 Test         
        mean    std      mean    std    mean    std  
154    0.863  0.024      0.02  0.001   0.974  0.002

<IPython.core.display.Javascript object>

In [19]:
pattern = r"One-Hot"

features_filter = df_nn["Features"].str.contains(
    pattern,
    regex=True,
)

idx_min = df_nn[features_filter].round(3)["RMSE Test", "mean"].idxmin()

df_nn.loc[[idx_min]].round(3)

Model                                  Features Cross Validation  \
                                                                              
494  Transformer  Chemical + Mineralogical + CS7 + One-Hot   Repeated KFold   

           Cross Validation Params RMSE Test        MAE Test        MAPE Test  \
                                        mean    std     mean    std      mean   
494  {"N_Splits": 5, "Repeats": 3}     1.048  0.104    0.824  0.102     0.019   

           R2 Test        
       std    mean   std  
494  0.002   0.956  0.01

<IPython.core.display.Javascript object>

# Trees

In [26]:
trees = ["Decision Tree", "Random Forest", "XGBoost"]

df_trees = df[df[("Model", "")].isin(trees)][cols].copy()
df_trees = df_trees[df_trees["Cross Validation"] != "Out of time Split"]

<IPython.core.display.Javascript object>

In [27]:
idx_min = df_trees[df_trees["Features"] == "Chemical"]["RMSE Test", "mean"].idxmin()

df_trees.drop(["Cross Validation Params"], axis=1).loc[[idx_min]].round(3)

/tmp/ipykernel_1879735/3648671257.py:3: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_trees.drop(["Cross Validation Params"], axis=1).loc[[idx_min]].round(3)


Model  Features Cross Validation RMSE Test        MAE Test  \
                                                   mean    std     mean   
423  Random Forest  Chemical   Repeated KFold     1.671  0.119    1.142   

           MAPE Test        R2 Test         
       std      mean    std    mean    std  
423  0.039     0.027  0.001   0.942  0.008

<IPython.core.display.Javascript object>

In [28]:
idx_min = df_trees[df_trees["Features"] == "Chemical + Properties CS Less"][
    "RMSE Test", "mean"
].idxmin()

df_trees.drop(["Cross Validation Params"], axis=1).loc[[idx_min]].round(3)

/tmp/ipykernel_1879735/3559523472.py:5: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_trees.drop(["Cross Validation Params"], axis=1).loc[[idx_min]].round(3)


Model                       Features Cross Validation RMSE Test  \
                                                                        mean   
547  Random Forest  Chemical + Properties CS Less   Repeated KFold     1.344   

           MAE Test        MAPE Test        R2 Test         
       std     mean    std      mean    std    mean    std  
547  0.071    1.026  0.037     0.024  0.001   0.963  0.004

<IPython.core.display.Javascript object>

In [29]:
exclude = r"(?!.*One-Hot)"
pattern1 = r"Chemical \+ CS*"
pattern2 = r"Chemical \+ Physical"
pattern = "(" + pattern1 + "|" + pattern2 + ")" + exclude

# x = df_nn["Features"] != "Chemical + Mineralogical + Physical"
features_filter = df_trees["Features"].str.contains(
    pattern,
    regex=True,
)

idx_min = df_trees[features_filter].round(3)["RMSE Test", "mean"].idxmin()

df_trees.drop(["Cross Validation Params"], axis=1).loc[[idx_min]].round(3)

/tmp/ipykernel_1879735/1286850631.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  features_filter = df_trees["Features"].str.contains(
/tmp/ipykernel_1879735/1286850631.py:14: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_trees.drop(["Cross Validation Params"], axis=1).loc[[idx_min]].round(3)


Model             Features Cross Validation RMSE Test         \
                                                              mean    std   
443  Random Forest  Chemical + Physical   Repeated KFold     1.077  0.027   

    MAE Test        MAPE Test        R2 Test         
        mean    std      mean    std    mean    std  
443    0.848  0.026      0.02  0.001   0.976  0.001

<IPython.core.display.Javascript object>

In [31]:
# idx_min = df_trees[df_trees["Features"] == "Chemical + Mineralogical + One-Hot"][
#     "RMSE Test", "mean"
# ].idxmin()

# df_trees.loc[[idx_min]].round(3)

<IPython.core.display.Javascript object>

In [24]:
pattern = r"One-Hot"

features_filter = df_trees["Features"].str.contains(
    pattern,
    regex=True,
)

idx_min = df_trees[features_filter].round(3)["RMSE Test", "mean"].idxmin()

df_trees.loc[[idx_min]].round(3)

Model                                  Features  \
                                                               
424  Random Forest  Chemical + Mineralogical + CS3 + One-Hot   

               Cross Validation  \
                                  
424  Blocking Time Series Split   

                              Cross Validation Params RMSE Test         \
                                                           mean    std   
424  {"N_Splits": 5, "Repeats": 1, "train_size": 0.8}     0.949  0.217   

    MAE Test        MAPE Test        R2 Test         
        mean    std      mean    std    mean    std  
424    0.709  0.138     0.016  0.003   0.962  0.016

<IPython.core.display.Javascript object>